# Imports

In [57]:
import pandas as pd
import numpy as np

In [58]:
def reduce_df(df:pd.DataFrame):
    # df[df['area']==0.05]
    return df.groupby('metric').aggregate('mean')[['comp', 'suff']].reset_index()

int_metric_map = {
    'electricity': ['mae'],
    'traffic': ['mae'],
    'mimic_iii': ['auc']
}

test_metric_map = {
    'electricity': ['mae', 'mse'],
    'traffic': ['mae', 'mse'],
    'mimic_iii': ['auc', 'accuracy']
}

datasets = ['electricity', 'traffic', 'mimic_iii']
models = ['DLinear', 'MICN', 'SegRNN', 'iTransformer']
attr_methods = [
    'feature_ablation', 'augmented_occlusion', 
    'integrated_gradients', 'gradient_shap', 
    'feature_permutation', 'winIT', 'tsr' ,'wtsr'
]

short_form = {
    'feature_ablation': 'FA',
    'occlusion':'FO',
    'augmented_occlusion': 'AFO',
    'feature_permutation': 'FP',
    'winIT': 'WinIT',
    'tsr':'TSR',
    'wtsr': 'WinTSR',
        'gradient_shap': 'GS',
    'integrated_gradients': 'IG'
}
NUM_ITERATIONS = 3

# Summarize

In [59]:
results = []
for dataset in datasets:
    for attr_method in attr_methods:
        for metric in int_metric_map[dataset]:
            for model in models:
                for itr_no in range(1, NUM_ITERATIONS+1):
                    df = pd.read_csv(f'results/{dataset}_{model}/{itr_no}/{attr_method}.csv')
                    df = reduce_df(df)
                    comp, suff= df[df['metric']==metric][['comp', 'suff']].values[0]
                    
                    results.append([
                        dataset, attr_method, metric, model, itr_no, comp, suff
                    ])

result_df = pd.DataFrame(results, columns=['dataset', 'attr_method', 'metric', 'model', 'itr_no', 'comp', 'suff'])
result_df.head(3)

,dataset,attr_method,metric,model,itr_no,comp,suff
0,electricity,feature_ablation,mae,DLinear,1,10.860362,12.558042
1,electricity,feature_ablation,mae,DLinear,2,10.433986,12.210812
2,electricity,feature_ablation,mae,DLinear,3,10.852448,12.406176


In [60]:
result_df = result_df.groupby(['dataset', 'attr_method', 'metric', 'model'])[['comp', 'suff']].mean().reset_index()
result_df = result_df[result_df['metric'].isin(['mae', 'auc'])]

selected = result_df['metric'].isin(['auc', 'accuracy'])
result_df.loc[selected, ['comp', 'suff']] = 1 - result_df[selected][['comp', 'suff']]

In [61]:
result_df['comp_rank'] = result_df.groupby(['dataset', 'metric', 'model'])['comp'].rank(ascending=False)
result_df['suff_rank'] = result_df.groupby(['dataset', 'metric', 'model'])['suff'].rank(ascending=True)
result_df.groupby(['dataset', 'metric', 'attr_method'])[['comp_rank', 'suff_rank']].mean().reset_index()

,dataset,metric,attr_method,comp_rank,suff_rank
0,electricity,mae,augmented_occlusion,5.25,4.00
1,electricity,mae,feature_ablation,3.50,3.00
2,electricity,mae,feature_permutation,7.75,6.50
3,electricity,mae,gradient_shap,7.00,7.00
4,electricity,mae,integrated_gradients,3.00,7.50
5,electricity,mae,tsr,2.25,2.00
6,electricity,mae,winIT,6.00,5.00
7,electricity,mae,wtsr,1.25,1.00
8,mimic_iii,auc,augmented_occlusion,5.50,2.50
9,mimic_iii,auc,feature_ablation,2.25,3.50


In [62]:
result_df.groupby(['dataset', 'metric', 'attr_method'])[['comp_rank', 'suff_rank']].std().round(1).reset_index()

,dataset,metric,attr_method,comp_rank,suff_rank
0,electricity,mae,augmented_occlusion,0.5,0.0
1,electricity,mae,feature_ablation,0.6,0.0
2,electricity,mae,feature_permutation,0.5,1.0
3,electricity,mae,gradient_shap,0.8,0.0
4,electricity,mae,integrated_gradients,1.4,1.0
5,electricity,mae,tsr,0.5,0.0
6,electricity,mae,winIT,0.8,0.0
7,electricity,mae,wtsr,0.5,0.0
8,mimic_iii,auc,augmented_occlusion,1.3,1.9
9,mimic_iii,auc,feature_ablation,1.0,1.0


In [63]:
df = pd.concat([
    result_df.drop(columns='suff_rank').rename(columns={'comp_rank': 'rank'}), 
    result_df.drop(columns='comp_rank').rename(columns={'suff_rank': 'rank'})
], axis=0)

In [64]:
df.groupby(['dataset', 'metric', 'attr_method'])['rank'].mean().round(1).reset_index()

,dataset,metric,attr_method,rank
0,electricity,mae,augmented_occlusion,4.6
1,electricity,mae,feature_ablation,3.2
2,electricity,mae,feature_permutation,7.1
3,electricity,mae,gradient_shap,7.0
4,electricity,mae,integrated_gradients,5.2
5,electricity,mae,tsr,2.1
6,electricity,mae,winIT,5.5
7,electricity,mae,wtsr,1.1
8,mimic_iii,auc,augmented_occlusion,4.0
9,mimic_iii,auc,feature_ablation,2.9


In [65]:
df.groupby(['dataset', 'metric', 'attr_method'])['rank'].std().round(2).reset_index()

,dataset,metric,attr_method,rank
0,electricity,mae,augmented_occlusion,0.74
1,electricity,mae,feature_ablation,0.46
2,electricity,mae,feature_permutation,0.99
3,electricity,mae,gradient_shap,0.53
4,electricity,mae,integrated_gradients,2.66
5,electricity,mae,tsr,0.35
6,electricity,mae,winIT,0.76
7,electricity,mae,wtsr,0.35
8,mimic_iii,auc,augmented_occlusion,2.20
9,mimic_iii,auc,feature_ablation,1.13
